## Import packages

In [85]:
import random
import math

## Load cities

In [86]:
def load_cities():
    cities = []
    f = open('citiesLocation.txt')
    for line in f.readlines():
        pos = line.split()
        cities.append([float(pos[0]),float(pos[1])])
    return cities

## Initial chromosome

In [87]:
def init_ch(cities):
    route = list(range(len(cities)))
    return random.sample(route, len(route))

## Generate Population

In [88]:
def init_population(cities, pop_size = 1000):
    population = [init_ch(cities) for _ in range(pop_size)]
    return population

## Calculate fitness

In [89]:
def fitness(ch, cities):
    total_distance = 0
    for i in range(len(ch) - 1):
        A_pos = cities[ch[i]]
        B_pos = cities[ch[i+1]]
        d = math.sqrt(math.pow(A_pos[0] - B_pos[0], 2) + math.pow(A_pos[1] - B_pos[1], 2)) 
        total_distance += d
    A_pos = cities[ch[0]]
    B_pos = cities[ch[-1]]
    d = math.sqrt(math.pow(A_pos[0] - B_pos[0], 2) + math.pow(A_pos[1] - B_pos[1], 2))
    total_distance += d
    return total_distance

## Selection

### Random Pick

In [90]:
def random_pick(population):
    fitnesses = [fitness(ch) for ch in population]
    max_fitness = max(fitnesses)
    fitness_probs = [(max_fitness - f) for f in fitnesses]
    total_fitness = sum(fitness_probs)
    fitness_probs = [f / total_fitness for f in fitness_probs]

    cumulative_probs = []
    cumulative_sum = 0
    for prob in fitness_probs:
        cumulative_sum += prob
        cumulative_probs.append(cumulative_sum)

    random_number = random.random()

    for i, cumulative_prob in enumerate(cumulative_probs):
        if random_number <= cumulative_prob:
            return population[i]
    return population[-1]

### Tournoment Selecton

In [91]:
def tournament_selection(population, cities, tournament_size = 5):
    tournament = random.sample(population, tournament_size)
    winner = min(tournament, key=lambda x: fitness(x, cities))
    return winner

## Crossover

### Single Point

In [92]:
def single_point_co(x,y):
    co_point = random.randint(1,len(x) - 1)
    offspring1 = x[:co_point] + y[co_point:]
    offspring2 = y[:co_point] + x[co_point:]

    return offspring1, offspring2

### Partially Mapped Crossover (PMX)

In [93]:
def pmx_co(x,y):
    n = len(x)
    pos1, pos2 = sorted(random.sample(range(1,n), k = 2))

    offspring1 = x[:]
    offspring2 = y[:]

    offspring1[pos1:pos2+1] = y[pos1:pos2+1]
    offspring2[pos1:pos2+1] = x[pos1:pos2+1]

    map1 = {y[i]: x[i] for i in range(pos1, pos2+1)}
    map2 = {x[i]: y[i] for i in range(pos1, pos2+1)}
    for i in range(n):
        if i < pos1 or i > pos2:
            if offspring1[i] not in y[pos1:pos2+1]:
                continue
            value = offspring1[i]
            while value in map1:
                value = map1[value]
            offspring1[i] = value
    
    for i in range(n):
        if i < pos1 or i > pos2:
            if offspring2[i] not in x[pos1:pos2+1]:
                continue
            value = offspring2[i]
            while value in map2:
                value = map2[value]
            offspring2[i] = value
    
    return offspring1, offspring2

### Order Crossover (OX)

In [94]:
def ox_co(x,y):
    n = len(x)
    pos1, pos2 = sorted(random.sample(range(n), k = 2))
    offspring1 = [None] * n
    offspring2 = [None] * n

    offspring1[pos1:pos2+1] = y[pos1:pos2+1]
    offspring2[pos1:pos2+1] = x[pos1:pos2+1]

    k1, k2 = (pos2 + 1) % n, (pos2 + 1) % n
    for i in range(n):
        gene1 = x[(i + pos2 + 1) % n]

        if gene1 not in offspring1[pos1:pos2+1]:
            offspring1[k1] = gene1
            k1 = (k1 + 1) % n
        gene2 = y[(i + pos2 + 1) % n]
        if gene2 not in offspring2[pos1:pos2+1]:
            offspring2[k2] = gene2
            k2 = (k2 + 1) % n
    for i in range(n):
        if offspring1[i] == None:
            offspring1[i] = x[i]
        if offspring2[i] == None:
            offspring2[i] = y[i]
    return offspring1, offspring2

## Mutation

In [95]:
def random_swap_mutation(ch):
    pos1, pos2 = random.sample(range(len(ch)), k = 2)
    mutated_ch = ch[:]
    mutated_ch[pos1], mutated_ch[pos2] = mutated_ch[pos2], mutated_ch[pos1]
    return mutated_ch

## Random Restart

In [96]:
def random_restart(population, cities, restart_fraction=0.5):
    num_to_restart = int(len(population) * restart_fraction)
    newPopulation = population[:]
    newPopulation = sorted(newPopulation, key=lambda ch: fitness(ch,cities), reverse=True)
    for i in range(num_to_restart):
        newPopulation[i] = init_ch(cities)
    newPopulation = sorted(newPopulation, key=lambda ch: fitness(ch,cities), reverse=False)
    return newPopulation

## Genetic Algorithm

In [97]:
def genetic_algo(population, best_distance , patient, cities):
    current_best_distance  = min([fitness(ch,cities) for ch in population])

    if current_best_distance == best_distance:
        patient += 1
    else:
        patient = 0
        best_distance = current_best_distance

    if patient >= 20:
        population = random_restart(population, cities)
        patient = 0

    new_population = []
    for i in range(int(len(population) / 2)):
        x = tournament_selection(population,cities)
        y = tournament_selection(population,cities)
        offspring1, offspring2 = pmx_co(x,y)
        new_population.append(offspring1)
        new_population.append(offspring2)

    return new_population, patient, best_distance

In [98]:
cities = load_cities()
population = init_population(cities)
max_generations = 3000
best_distance = 2 * math.pow(10,7)
patient = 0

for generation in range(max_generations):
    population, patient, best_distance = genetic_algo(population, best_distance, patient, cities)
    if(generation % 10 == 0):
        print(f"Generation: {generation}\nBest fitness: {best_distance}")

fitnesses = [fitness(ch,cities) for ch in population]
bestCh = population[fitnesses.index(min(fitnesses))]
best_distance = min(fitnesses)
print(f"Answer: {bestCh}\n Best distance: {best_distance}")

Generation: 0
Best fitness: 138.9004810394787
Generation: 10
Best fitness: 100.90445503096103
Generation: 20
Best fitness: 75.58076250208063
Generation: 30
Best fitness: 75.57776525236525
Generation: 40
Best fitness: 75.57776525236525
Generation: 50
Best fitness: 75.57776525236525
Generation: 60
Best fitness: 75.57776525236525
Generation: 70
Best fitness: 75.57776525236525
Generation: 80
Best fitness: 75.57776525236525
Generation: 90
Best fitness: 75.57776525236525
Generation: 100
Best fitness: 75.57776525236525
Generation: 110
Best fitness: 75.57776525236525
Generation: 120
Best fitness: 75.57776525236525
Generation: 130
Best fitness: 75.57776525236525
Generation: 140
Best fitness: 75.57776525236525
Generation: 150
Best fitness: 75.57776525236525
Generation: 160
Best fitness: 75.57776525236525
Generation: 170
Best fitness: 75.57776525236525
Generation: 180
Best fitness: 75.57776525236525
Generation: 190
Best fitness: 75.57776525236525
Generation: 200
Best fitness: 75.57776525236525
Ge